# Data Cleaning

# Missibg Value Imputation Using Scikit-learn

# Different Strategy for different variables(Numerical & Categorical) with Scikit-learn

In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
train = pd.read_csv(r"D:\Data Cleaning\house-prices-advanced-regression-techniques\train.csv")
test = pd.read_csv(r"D:\Data Cleaning\house-prices-advanced-regression-techniques\test.csv")

In [3]:
print("Shape of train df =", train.shape)
print("Shape of test df =", test.shape)

Shape of train df = (1460, 81)
Shape of test df = (1459, 80)


In [4]:
X_train = train.drop(columns = "SalePrice", axis =1)
y_train = train["SalePrice"]
X_test = test.copy()

print("Shape of X_train = ", X_train.shape)
print("Shape of y_train = ", y_train.shape)
print("Shape of X_test = ", X_test.shape)

Shape of X_train =  (1460, 80)
Shape of y_train =  (1460,)
Shape of X_test =  (1459, 80)


In [5]:
isnull_sum = X_train.isnull().sum()

isnull_sum

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
Length: 80, dtype: int64

In [6]:
# finding the numerical variable which have missing value
num_var = X_train.select_dtypes(include=["int64","float64"]).columns

num_var_miss = [var for var in num_var if isnull_sum[var]>0]

In [7]:
num_var_miss

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [8]:
# finding the categorical variable which have missing value
cat_var = X_train.select_dtypes(include=["O"]).columns

cat_var_miss = [var for var in cat_var if isnull_sum[var]>0]

In [9]:
cat_var_miss

['Alley',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [10]:
num_var_mean = ["LotFrontage"]
num_var_median = ['MasVnrArea', 'GarageYrBlt']

cat_var_mode = ['Alley','MasVnrType','BsmtQual','BsmtCond','BsmtExposure',
                'BsmtFinType1','BsmtFinType2','Electrical','FireplaceQu']
cat_var_missing = ['GarageType','GarageFinish','GarageQual','GarageCond',
                   'PoolQC','Fence','MiscFeature']


In [14]:
num_var_mean_imputer = Pipeline(steps = [("imputer", SimpleImputer(strategy = "mean"))])
num_var_median_imputer = Pipeline(steps = [("imputer", SimpleImputer(strategy = "median"))])
cat_var_mode_imputer = Pipeline(steps = [("imputer", SimpleImputer(strategy = "most_frequent"))])
cat_var_missing_imputer = Pipeline(steps = [("imputer", SimpleImputer(strategy = "constant", fill_value = "missing"))])

In [15]:
preprocessor = ColumnTransformer(transformers = [("mean_imputer", num_var_mean_imputer, num_var_mean),
                                                ("median_imputer", num_var_median_imputer, num_var_median),
                                                ("mode_imputer",cat_var_mode_imputer, cat_var_mode),
                                                ("missing_imputer",cat_var_missing_imputer,cat_var_missing)])

In [16]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('mean_imputer',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['LotFrontage']),
                                ('median_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['MasVnrArea', 'GarageYrBlt']),
                                ('mode_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond',
                                  'BsmtExposure', 'BsmtFinType1',
                                  'BsmtFinType2', 'Electrical',
                                  'FireplaceQu']),
                                ('missing_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant'))]),
                                 ['GarageType', 'GarageFinish', 'GarageQual',
                                  'GarageCond', 'PoolQC', 'Fence',
                                  'MiscFeature'])])

In [17]:
preprocessor.transform

<bound method ColumnTransformer.transform of ColumnTransformer(transformers=[('mean_imputer',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['LotFrontage']),
                                ('median_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['MasVnrArea', 'GarageYrBlt']),
                                ('mode_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond',
                                  'BsmtExposure', 'BsmtFinType1',
                                  'BsmtFinType2', 'Electrical',
                                  'FireplaceQu']),
                                ('missing_im

In [18]:
preprocessor.named_transformers_["mean_imputer"].named_steps["imputer"].statistics_


array([70.04995837])

In [19]:
train["LotFrontage"].mean()

70.04995836802665

In [20]:
preprocessor.named_transformers_["mode_imputer"].named_steps["imputer"].statistics_


array(['Grvl', 'None', 'TA', 'TA', 'No', 'Unf', 'Unf', 'SBrkr', 'Gd'],
      dtype=object)

In [21]:
X_train_clean = preprocessor.transform(X_train)
X_test_clean = preprocessor.transform(X_test)

In [22]:
X_train_clean

array([[65.0, 196.0, 2003.0, ..., 'missing', 'missing', 'missing'],
       [80.0, 0.0, 1976.0, ..., 'missing', 'missing', 'missing'],
       [68.0, 162.0, 2001.0, ..., 'missing', 'missing', 'missing'],
       ...,
       [66.0, 0.0, 1941.0, ..., 'missing', 'GdPrv', 'Shed'],
       [68.0, 0.0, 1950.0, ..., 'missing', 'missing', 'missing'],
       [75.0, 0.0, 1965.0, ..., 'missing', 'missing', 'missing']],
      dtype=object)

In [23]:
preprocessor.transformers

[('mean_imputer',
  Pipeline(steps=[('imputer', SimpleImputer())]),
  ['LotFrontage']),
 ('median_imputer',
  Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))]),
  ['MasVnrArea', 'GarageYrBlt']),
 ('mode_imputer',
  Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent'))]),
  ['Alley',
   'MasVnrType',
   'BsmtQual',
   'BsmtCond',
   'BsmtExposure',
   'BsmtFinType1',
   'BsmtFinType2',
   'Electrical',
   'FireplaceQu']),
 ('missing_imputer',
  Pipeline(steps=[('imputer',
                   SimpleImputer(fill_value='missing', strategy='constant'))]),
  ['GarageType',
   'GarageFinish',
   'GarageQual',
   'GarageCond',
   'PoolQC',
   'Fence',
   'MiscFeature'])]

In [24]:
X_train_clean_miss_var = pd.DataFrame(X_train_clean, columns = num_var_mean + num_var_median + cat_var_mode + cat_var_missing)

In [25]:
X_train_clean_miss_var.head()

,LotFrontage,MasVnrArea,GarageYrBlt,Alley,MasVnrType,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Electrical,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PoolQC,Fence,MiscFeature
0,65.0,196.0,2003.0,Grvl,BrkFace,Gd,TA,No,GLQ,Unf,SBrkr,Gd,Attchd,RFn,TA,TA,missing,missing,missing
1,80.0,0.0,1976.0,Grvl,None,Gd,TA,Gd,ALQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,missing,missing,missing
2,68.0,162.0,2001.0,Grvl,BrkFace,Gd,TA,Mn,GLQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,missing,missing,missing
3,60.0,0.0,1998.0,Grvl,None,TA,Gd,No,ALQ,Unf,SBrkr,Gd,Detchd,Unf,TA,TA,missing,missing,missing
4,84.0,350.0,2000.0,Grvl,BrkFace,Gd,TA,Av,GLQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,missing,missing,missing


In [26]:
X_train_clean_miss_var.isnull().sum().sum()

0

In [27]:
train['Alley'].value_counts()

Grvl    50
Pave    41
Name: Alley, dtype: int64

In [28]:
X_train_clean_miss_var['Alley'].value_counts()

Grvl    1419
Pave      41
Name: Alley, dtype: int64

In [29]:
X_train_clean_miss_var['MiscFeature'].value_counts()

missing    1406
Shed         49
Gar2          2
Othr          2
TenC          1
Name: MiscFeature, dtype: int64